### AWS Braket Test

In [1]:
from braket.circuits import Circuit
from braket.devices import LocalSimulator
from braket.aws import AwsDevice

from braket.tracking import Tracker

#### AWS Devices

In [7]:
# 1) Local Simulator

local_simulator = LocalSimulator()

bell = Circuit().h(0).cnot(0, 1)

counts = local_simulator.run(bell, shots=100).result().measurement_counts

counts

Counter({'00': 51, '11': 49})

In [4]:
# SV1

cost_tracker = Tracker().start()

sv1_device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/sv1')  

counts = sv1_device.run(bell, shots=100).result().measurement_counts

counts

Counter({'11': 56, '00': 44})

In [5]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 100,
  'tasks': {'COMPLETED': 1},
  'execution_duration': datetime.timedelta(microseconds=3000),
  'billed_execution_duration': datetime.timedelta(seconds=3)}}

In [6]:
# 3) Batch SV1

circuits = [bell for _ in range(5)]

# batch_results = sv1_device.run_batch(task_specifications=circuits, shots=100)

In [ ]:
print(batch.results()[0].measurement_counts)  # The result of the first task in the batch

#### PennyLane

In [4]:
def filter_circuit(pixel_values, random_layer_parameters):
       
    qubits = list(range(len(pixel_values)))

    for qubit, pixel_value in enumerate(pixel_values):     

        theta = jnp.pi * pixel_value        
        qml.RY(theta, wires=qubit)        
    
    RandomLayers(random_layer_parameters, wires=qubits)

    measurement_result = [qml.expval(qml.PauliZ(qubit)) for qubit in qubits]
    
    return measurement_result

In [92]:
QUBITS_COUNT = 27
LAYERS_COUNT = 1

pixel_values = np.random.random(size=(QUBITS_COUNT,), requires_grad=False)

random_layer_parameters = np.random.random(size=(LAYERS_COUNT, QUBITS_COUNT), requires_grad=True)

qubits = list(range(len(pixel_values)))

pixel_values

tensor([0.11483682, 0.61062004, 0.28863055, 0.58123822, 0.15436272,
        0.4811401 , 0.53258943, 0.05182354, 0.33660428, 0.13441468,
        0.06337497, 0.98996023, 0.32235384, 0.80987445, 0.25464065,
        0.68150272, 0.76022786, 0.59563874, 0.47157619, 0.41184091,
        0.34886827, 0.92952914, 0.83061941, 0.96502691, 0.12429722,
        0.73086748, 0.93834046], requires_grad=False)

In [93]:
random_layer_parameters

tensor([[0.18123307, 0.06649627, 0.74112065, 0.57447311, 0.84182878,
         0.13977238, 0.79526731, 0.20162732, 0.16365594, 0.1642658 ,
         0.81457472, 0.66519722, 0.52306542, 0.35883048, 0.87720054,
         0.39244511, 0.81659944, 0.43913491, 0.37694443, 0.46267979,
         0.30137787, 0.74760938, 0.50272039, 0.2322127 , 0.89957457,
         0.38389122, 0.54355286]], requires_grad=True)

In [94]:
filter_circuit(pixel_values, random_layer_parameters)

[expval(PauliZ(wires=[0])),
 expval(PauliZ(wires=[1])),
 expval(PauliZ(wires=[2])),
 expval(PauliZ(wires=[3])),
 expval(PauliZ(wires=[4])),
 expval(PauliZ(wires=[5])),
 expval(PauliZ(wires=[6])),
 expval(PauliZ(wires=[7])),
 expval(PauliZ(wires=[8])),
 expval(PauliZ(wires=[9])),
 expval(PauliZ(wires=[10])),
 expval(PauliZ(wires=[11])),
 expval(PauliZ(wires=[12])),
 expval(PauliZ(wires=[13])),
 expval(PauliZ(wires=[14])),
 expval(PauliZ(wires=[15])),
 expval(PauliZ(wires=[16])),
 expval(PauliZ(wires=[17])),
 expval(PauliZ(wires=[18])),
 expval(PauliZ(wires=[19])),
 expval(PauliZ(wires=[20])),
 expval(PauliZ(wires=[21])),
 expval(PauliZ(wires=[22])),
 expval(PauliZ(wires=[23])),
 expval(PauliZ(wires=[24])),
 expval(PauliZ(wires=[25])),
 expval(PauliZ(wires=[26]))]

In [95]:
cost_tracker = Tracker().start()

In [96]:
# diff_method = 'device'
diff_method = 'parameter-shift'

dev = qml.device(
    name="braket.aws.qubit",
    device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    wires=qubits,
    shots=1000,
)

In [101]:
filter_q_node = qml.QNode(func=filter_circuit, device=dev, diff_method=diff_method)

filter_q_node(pixel_values, random_layer_parameters)

tensor([ 0.924,  0.056,  0.58 , -0.264,  0.908, -0.372, -0.072,  0.704,
        -0.3  ,  0.896, -0.064,  0.316, -0.568, -0.828, -0.16 , -0.516,
        -0.704, -0.336,  0.024,  0.292, -0.42 , -0.984, -0.248, -0.12 ,
         0.284, -0.948, -0.824], requires_grad=True)

In [102]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 1000,
  'tasks': {'COMPLETED': 2},
  'execution_duration': datetime.timedelta(seconds=6, microseconds=979000),
  'billed_execution_duration': datetime.timedelta(seconds=6, microseconds=979000)}}

In [98]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 500,
  'tasks': {'COMPLETED': 1},
  'execution_duration': datetime.timedelta(seconds=3, microseconds=548000),
  'billed_execution_duration': datetime.timedelta(seconds=3, microseconds=548000)}}

In [56]:
# cost_tracker.tracked_resources()
# cost_tracker.qpu_tasks_cost()

cost_tracker.simulator_tasks_cost()

Decimal('0.0037500000')

In [22]:
cost_tracker = Tracker().start()

In [23]:
# Batching

diff_method = 'parameter-shift'

dev_batched = qml.device(
    name="braket.aws.qubit",
    device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    wires=qubits,
    shots=1000,
    parallel=True,
)

In [24]:
filter_q_node_batched = qml.qnode(dev_batched, diff_method=diff_method)(filter_circuit)

filter_q_node_batched(pixel_values, random_layer_parameters)

tensor([ 0.924, -0.134], requires_grad=True)

In [25]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 1000,
  'tasks': {'COMPLETED': 1},
  'execution_duration': datetime.timedelta(microseconds=66000),
  'billed_execution_duration': datetime.timedelta(seconds=3)}}

### QNodeCollection

In [1]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 20

dev = qml.device('lightning.qubit', wires=n_qubits)

In [3]:
@qml.qnode(dev)
def circuit():
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    return qml.expval(qml.PauliZ(0))

# one circuit execution
%time circuit() 

CPU times: user 33 ms, sys: 10.5 ms, total: 43.5 ms
Wall time: 40.8 ms


tensor(0., requires_grad=True)

In [10]:
circuits_count = 50

circuits = qml.QNodeCollection([circuit] * circuits_count)

In [11]:
# two circuit executions
%time circuits(parallel=True)

CPU times: user 1.62 s, sys: 66.3 ms, total: 1.69 s
Wall time: 1.5 s


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
from braket.tracking import Tracker

cost_tracker = Tracker().start()

qubits = list(range(n_qubits))

sv1_batched = qml.device(
    name="braket.aws.qubit",
    device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    wires=qubits,
    shots=1000,
    parallel=True,
)

In [34]:
@qml.qnode(sv1_batched)
def sv1_batched_circuit():
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    return qml.expval(qml.PauliZ(0))

In [24]:
# one circuit execution
%time sv1_batched_circuit() 

CPU times: user 270 ms, sys: 0 ns, total: 270 ms
Wall time: 8.09 s


tensor(-0.038, requires_grad=True)

In [25]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 1000,
  'tasks': {'COMPLETED': 1},
  'execution_duration': datetime.timedelta(microseconds=25000),
  'billed_execution_duration': datetime.timedelta(seconds=3)}}

In [35]:
circuits_count = 3

sv1_batched_circuits = qml.QNodeCollection([sv1_batched_circuit] * circuits_count)

In [37]:
sv1_batched_circuits()

array([ 0.006, -0.006,  0.022])

In [38]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 3000,
  'tasks': {'COMPLETED': 3},
  'execution_duration': datetime.timedelta(microseconds=177000),
  'billed_execution_duration': datetime.timedelta(seconds=9)}}